# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Hands-On LangChain for LLM Applications Development: Output Parsing </center>

***

When developing a complex application with a Language Model (LLM), it’s common to specify the desired output format, such as JSON, and designate particular keys for organizing the data. 

Let’s consider the chain of thought reasoning method as an illustrative example. In this method, the LLM’s thinking process is represented by distinct stages: “thought” indicates the reasoning process, “action” denotes the subsequent action taken, and “observation” reflects the learning acquired from that action, and so forth. By crafting a prompt that directs the LLM to utilize these specific keywords (thought, action, observation), we can effectively guide its cognitive process. 

In this article, we will cover coupling the prompt with a parser that allows for the extraction of text associated with certain keywords from the LLM’s output. This combined approach offers a streamlined means of specifying input for the LLM and accurately interpreting its output.


# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Parsing Output Using LangChain Prompt Templates </b></div>


Let's start with an example to clarify the output parsing concept. Let’s take a look at how you can have an LLM output JSON, and use LangChain to parse that output. 

In this example, we will extract information from a product review and format that output in a JSON format. Here’s an example of how you would like the output formatted. Technically, this is a Python dictionary, where whether or not the product is a gift, maps to false, the number of days it took to deliver was five, and the price value was pretty affordable. So this is one example of a desired output:



In [1]:
x = {
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [2]:
x.get("price_value")

'pretty affordable!'

Here is an example of a customer review, as well as a template and we want to get the output as JSON. 

In [3]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

So, the review template asks the LLM to take as input a customer review extract these three fields and then format the output as JSON with the following keys. 

So here’s how you can wrap this in LangChain. First, we will import the chat prompt template. Then we will have the prompt templates created from the review template up on top. 

In [4]:
!pip install langchain
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.9/351.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 91.8 MB/s eta 0:00:00
  Attempting uninstall: zstandard
    Found existing installation: zstandard 0.22.0
    Uninstalling zstandard-0.22.0:
      Successfully uninstalled zstandard-0.22.0
  Attempting uninstall: typing-extensions
 

In [5]:
!pip install openai -q

In [6]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


Let’s create the messages to pass to the OpenAI, endpoint. Create the OpenAI endpoint, call that endpoint, and then let’s print out the response. I encourage you to pause the video and run the code.

In [7]:
from openai import OpenAI
import openai
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_api")
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

llm_model = "gpt-4o-mini"

In [8]:
from langchain_community.chat_models import ChatOpenAI

messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model, openai_api_key=openai.api_key)
response = chat(messages)
print(response.content)

/tmp/ipykernel_18/1190519107.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(temperature=0.0, model=llm_model, openai_api_key=openai.api_key)
/tmp/ipykernel_18/1190519107.py:5: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat(messages)


```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there", "I think it's worth it for the extra features"]
}
```


It says the gift is true, the delivery day is 2, and the price value also looks pretty accurate. But note that if we check the type of the response, this is a string. So it looks like JSON and has key-value pairs, but it’s not a dictionary. This is just one long string.



In [9]:
type(response.content)

str

So what I’d like to do is go to the response content and get the value from the gift key which should be true, but if I run this, this should generate an error because, well, this is a string. This is not a Python dictionary.

In [10]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string

# response.content.get('gift')

In [11]:
response.content[5]

'o'

Let’s see how we will use LangChain’s parser to do this.

# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Parse the LLM Output String into a Python Dictionary </b></div>


To parse the output into json format and make its type Python dict not string we will start with importing the response schema and structured output parser from LangChain. 

I am going to tell it what I wanted to parse by specifying these response schemas. So the gif schema, delivery days schema, and price value schema, here are their description.

In [12]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

Now that I’ve specified the schema for these LangChain can give you the prompt itself by having the output parser tell you what instructions it wants you to send to the LLM. So if I were to print format instructions.

In [13]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


This is a precise set of instructions for the LLM that will cause it to generate an output that the output parser can process. So here’s the new review template and the review template includes the format instructions that LangChain generated.

We can create a prompt from the review template too, and then create the messages that will pass to the OpenAI endpoint. If you want, you can take a look at the actual prompt, which gives the instructions to extract the fields gift, delivery days, and price value, here’s the text, and then here are the formatting instructions.

In [14]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [15]:
messages

[HumanMessage(content='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the 

Finally, if we call the OpenAI endpoint, let’s take a look at what response we got. It is now this, and now if we use the output parser that we created earlier, you can then parse this into an output dictionary, which if I print, looks like this. 

Notice that this is of type dictionary, not a string, which is why I can now extract the value associated with the key gift and get true, or the value associated with delivery days and get two, or you can also extract the value associated with price value. So this is a nifty way to take your LLM output and parse it into a Python dictionary, to make the output easier to use in downstream processing.

In [16]:
# response = chat(messages)
# output_dict = output_parser.parse(response.content)
# output_dict


# <div style="box-shadow: rgba(240, 46, 170, 0.4) -5px 5px inset, rgba(240, 46, 170, 0.3) -10px 10px inset, rgba(240, 46, 170, 0.2) -15px 15px inset, rgba(240, 46, 170, 0.1) -20px 20px inset, rgba(240, 46, 170, 0.05) -25px 25px inset; padding:20px; font-size:30px; font-family: consolas; display:fill; border-radius:15px; color: rgba(240, 46, 170, 0.7)"> <b> ༼⁠ ⁠つ⁠ ⁠◕⁠‿⁠◕⁠ ⁠༽⁠つ Thank You!</b></div>
​
<p style="font-family:verdana; color:rgb(34, 34, 34); font-family: consolas; font-size: 16px;"> 💌 Thank you for taking the time to read through my notebook. I hope you found it interesting and informative. If you have any feedback or suggestions for improvement, please don't hesitate to let me know in the comments. <br><br> 🚀 If you liked this notebook, please consider upvoting it so that others can discover it too. Your support means a lot to me, and it helps to motivate me to create more content in the future. <br><br> ❤️ Once again, thank you for your support, and I hope to see you again soon!</p>